### 평가

In [4]:
import pandas as pd

In [14]:
result_file = "evaluation_results/gpt4o-judge_result_20250220_q_QnA_car_sample_14_test.xlsx"
final_df = pd.read_excel(result_file)

In [15]:
print("\n=== 평가 결과 요약 ===")

# 평가할 메트릭 정의
metrics = {
    'correctness_score': lambda x: (x == True).mean(),
    'relevance_score': lambda x: (x == True).mean(),
    'grounded_score': lambda x: (x == True).mean(),
    'retrieval_relevance_score': lambda x: (x == True).mean(),
    'precision': 'mean',
    'recall': 'mean',
    'f1': 'mean',
    'rougeL': 'mean',
    'bleu': 'mean',
    'fail_rate': 'mean'
}

# groupby를 사용하여 한 번에 계산
result_df = final_df.groupby('method').agg(metrics).reset_index()

# 결과 저장
result_df.to_excel("output.xlsx", index=False)

# 결과 출력 (선택사항)
print("\n메소드별 평가 결과:")
print(result_df.round(4))  # 소수점 4자리까지 표시


=== 평가 결과 요약 ===

메소드별 평가 결과:
                                              method  correctness_score  \
0         hybrid_api+bm25_HyDE_full_page_no_reranker             0.3571   
1         hybrid_sts+bm25_HyDE_full_page_no_reranker             0.5000   
2           hybrid_sts+bm25_HyDE_full_page_rerank_20             0.5714   
3         hybrid_sts+bm25_None_full_page_no_reranker             0.5000   
4           hybrid_sts+bm25_None_full_page_rerank_20             0.4286   
5  hybrid_sts+bm25_QueryRewrite_full_page_no_rera...             0.6429   
6   hybrid_sts+bm25_QueryRewrite_full_page_rerank_20             0.5000   

   relevance_score  grounded_score  retrieval_relevance_score  precision  \
0           0.6429          0.4286                     0.9286     0.1477   
1           0.5714          0.4286                     0.7143     0.2050   
2           0.7143          0.4286                     0.7857     0.0942   
3           0.6429          0.5000                     0.7857   